In [1]:
import os
os.chdir("../..")
import dotenv


dotenv.load_dotenv(override=True)

print(os.getcwd(), os.getenv("HF_HOME"))

import json
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
from hashlib import sha256
from pathlib import Path

import datasets
import matplotlib.pyplot as plt
import pandas as pd
import tqdm

/data1/xhuan192/codes/med-sipf cache/


/data1/xhuan192/misc/miniconda3/envs/xiaoke-med_sipf-1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.getcwd()

'/data1/xhuan192/codes/med-sipf'

In [3]:
def remove_boxed(s, left="\\boxed"):
    original_s = s
    left = left + "{"
    try:
        assert s[: len(left)] == left
        assert s[-1] == "}"
        answer = s[len(left) : -1]
        if "=" in answer:
            answer = answer.split("=")[-1].lstrip(" ")
        return answer
    except Exception:
        return original_s


def last_boxed_only_string(string, left="\\boxed"):
    idx = string.rfind("\\boxed")
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return string
    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1

    if right_brace_idx is None:
        retval = string
    else:
        retval = string[idx : right_brace_idx + 1]

    return retval


def extract_answer(answer):
    # Try to extract content inside \boxed{}
    answer = remove_boxed(last_boxed_only_string(answer))
    return remove_boxed(last_boxed_only_string(answer, "\\text"), "\\text")


def eval_sample(x, prefix=""):
    model_answer_string = x["distilled_answer_string"]
    answer_letter = x["answer_letter"]
    answer_idx = x["answer_idx"]
    answer_string = x["answer_string"]

    extracted_answer_string = extract_answer(model_answer_string)

    try:
        if extracted_answer_string.strip().lower() == answer_string.strip().lower():
            correct = True
        elif (
            extracted_answer_string[0].strip().lower() == answer_letter.strip().lower()
        ):
            correct = True
        else:
            correct = False
    except Exception as e:
        print(f"Error: {e}\n{model_answer_string}\n{extracted_answer_string}")
        correct = False
    return {
        f"{prefix}extracted_answer_string": extracted_answer_string,
        f"{prefix}model_answer_string": model_answer_string,
        f"{prefix}correct": correct,
    }


In [4]:
def eval_correctness(path, split, prefix):
    dataset = datasets.load_dataset(path,split=split)

    mapped_dataset = dataset.map(partial(eval_sample, prefix=prefix), keep_in_memory=True, remove_columns=dataset.column_names)
    return mapped_dataset

path_list = ["mmqm/m196k-dedup-decon-qwen-7b", "mmqm/m196k-dedup-decon-qwen-32b"]
prefix_list = ["qwen_7b-", "qwen_32b-"]
split="train"

eval_correctness_list = []

for path, prefix in zip(path_list, prefix_list):

    mapped_dataset = eval_correctness(path, split, prefix)
    eval_correctness_list.append(mapped_dataset)

    correct_list=mapped_dataset[f"{prefix}correct"]
    print(f"path: {path}")
    print(f"correct: {sum(correct_list)}")
    print(f"total: {len(correct_list)}")
    print(f"accuracy: {sum(correct_list)/len(correct_list)}")

Map:   8%|▊         | 16035/192023 [00:01<00:15, 11496.47 examples/s]

Error: string index out of range
\boxed{}None of the options A, B, C, or D are primarily associated with cough. These medications are blood pressure medications and none of them are direct treatments for cough. If a cough is a side effect, it would depend on the specific medication and its mechanism of action. However, based on the given options, none are directly linked to cough.



Map:  10%|▉         | 19043/192023 [00:01<00:14, 11772.05 examples/s]

Error: string index out of range
\boxed{} 

None of the options provided are correct for the cause of reactive arthritis. Reactive arthritis is typically triggered by an immune response to a previous infection, most often with bacteria from the gastrointestinal or urinary tract, such as certain strains of *Shigella*, *Yersinia*, *Campylobacter*, or *Chlamydia*. However, the specific strains listed in the options (Shigella flexneri, Shigella boydii, Shigella shiga, and Shigella dysenteriae) are indeed associated with infectious colitis, which can sometimes lead to reactive arthritis in susceptible individuals. Therefore, while the question provides the correct bacteria, it does not identify the correct strain or directly state that reactive arthritis is caused by these bacteria.

Error: string index out of range
\boxed{}

Error: string index out of range
\boxed{} To make an accurate diagnosis, I would need more specific information about the clinical presentation and symptoms. However, 

Map:  12%|█▏        | 22744/192023 [00:02<00:13, 12119.99 examples/s]

...

Error: string index out of range
The question seems to be incomplete or lacks context about the "highlighted bone". Without specific details about which bone is being referred to, it's not possible to accurately determine which of the provided rules (A, B, C, or D) is associated with it. However, based on the options given, none are commonly recognized across general medical or anatomical contexts, indicating the need for more specific information or context to offer a precise answer.

Given the information, a direct answer cannot be provided, but if we were to select an option purely based on available choices without context, it wouldn't be informative or correct. Thus, I can't accurately provide a response to this question as it stands.

Therefore, I can't enclose a meaningful answer in \boxed{}. More specific information is required.



Map:  98%|█████████▊| 188900/192023 [00:15<00:00, 12011.05 examples/s]

Error: string index out of range
The listed options A through D do not correctly relate to the diagnosis of hypokalemic periodic paralysis. Hypokalemic periodic paralysis is a rare genetic disorder characterized by episodic muscle weakness and occasional flaccid paralysis associated with a drop in potassium levels (hypokalemia) in the blood.

- **Rhinophyma** (Option A) is a skin condition involving the enlargement of the nose due to excess tissue often related to rosacea.
- **Pheochromocytoma** (Option B) is a tumor commonly found in the adrenal glands that can cause episodes of high blood pressure and other symptoms but is not linked to muscle weakness or hypokalemia.
- **Grave's disease** (Option C), an autoimmune disorder that leads to overactivity of the thyroid gland (hyperthyroidism), may have muscle weakness as a symptom, but it is not specifically associated with hypokalemic periodic paralysis or hypokalemia.
- **Buphthalmos** (Option D) is a condition characterized by an abno

Map: 100%|██████████| 192023/192023 [00:16<00:00, 11908.82 examples/s]

path: mmqm/m196k-dedup-decon-qwen-32b
correct: 140286
total: 192023
accuracy: 0.7305687339537451


path: mmqm/m196k-dedup-decon-qwen-7b
correct: 117754
total: 192023
accuracy: 0.6132286236544581

path: mmqm/m196k-dedup-decon-qwen-32b
correct: 140286
total: 192023
accuracy: 0.7305687339537451

In [5]:
eval_correctness_list[0][0]

{'qwen_7b-extracted_answer_string': 'A. Nitrofurantoin',
 'qwen_7b-model_answer_string': '\\boxed{A. Nitrofurantoin}\n\nFor a pregnant woman presenting with symptoms of a urinary tract infection (UTI) such as burning upon urination, and considering she is at 22 weeks gestation, nitrofurantoin is generally considered the first-line treatment. It is safe during pregnancy and has minimal risk to the fetus. It is effective against the common pathogens that cause UTIs. The other antibiotics listed (doxycycline, ampicillin, and ceftriaxone) are either contraindicated or have a higher risk profile in pregnancy.',
 'qwen_7b-correct': True}

In [6]:
path="mmqm/m196k-dedup-decon"
split="train"

raw_dataset = datasets.load_dataset(path,split=split)
merged_dataset = datasets.concatenate_datasets([raw_dataset] + eval_correctness_list, axis=1)

In [7]:
def filter_correctness(x, prefix_list=["qwen_7b-", "qwen_32b-"]):
    for prefix in prefix_list:
        if x[f"{prefix}correct"]:
            return False
    return True

filtered_dataset = merged_dataset.filter(filter_correctness)

Filter: 100%|██████████| 192023/192023 [00:02<00:00, 72844.65 examples/s]


In [8]:
print(f"len(filtered_dataset): {len(filtered_dataset)}")
filtered_dataset[10]

len(filtered_dataset): 37816


{'answer_idx': 2,
 'source': 'GBaker/MedQA-USMLE-4-options',
 'metadata': "{'question': 'A healthy 22-year-old male participates in a research study you are leading to compare the properties of skeletal and cardiac muscle. You conduct a 3-phased experiment with the participant. In the first phase, you get him to lift up a 2.3 kg (5 lb) weight off a table with his left hand. In the second phase, you get him to do 20 burpees, taking his heart rate to 150/min. In the third phase, you electrically stimulate his gastrocnemius with a frequency of 50 Hz. You are interested in the tension and electrical activity of specific muscles as follows: Biceps in phase 1, cardiac muscle in phase 2, and gastrocnemius in phase 3. What would you expect to be happening in the phases and the respective muscles of interest?', 'answer': 'Increase of tension in all phases', 'options': {'A': 'Recruitment of small motor units at the start of experiments 1 and 2', 'B': 'Recruitment of large motor units followed by

In [9]:
filtered_dataset.push_to_hub("mmqm/m196k-dedup-decon-filter_easy")

Uploading the dataset shards: 100%|██████████| 1/1 [00:06<00:00,  6.00s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/mmqm/m196k-dedup-decon-filter_easy/commit/5a3887b1c7d1ad1d04b4215aa0b9c231dc76c86e', commit_message='Upload dataset', commit_description='', oid='5a3887b1c7d1ad1d04b4215aa0b9c231dc76c86e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mmqm/m196k-dedup-decon-filter_easy', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mmqm/m196k-dedup-decon-filter_easy'), pr_revision=None, pr_num=None)